![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)

## Prompt Engineering Challenge ##

In this notebook you will be challeanged to write your own prompts. 

## Before you start

Keep in mind that <br> 

💡 Prompt engineering is essentially the art and science of crafting well-structured and precise queries that enable AI models to provide accurate, informative and context-aware responses. So feel free to experiment with different prompts to find the one works the best for your use case!  

🎈 The default model defined in the helper function is `llama` You can choose another model supported by watsonx.ai and test the prompts. Read the [WML docs](https://ibm.github.io/watson-machine-learning-sdk/index.html) and find the model parameter you need to switch to the model of your choice. Yes, this is also an exercise about reading documentaion and python code:) Don't forget that you can adjust other model parameters such as decoding_method, max_new_tokens, min_new_tokens as well!

⏰ It should take you about 30-45 min to walk through the exercises self paced. 

**If you prefer to use the Prompt Lab, and not a notebook, you can simply copy the prompts into the Prompt Lab UI**

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependecies

In [1]:
!pip install ibm-watson-machine-learning --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.6 MB/s eta 0:00:0000:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.320
    Uninstalling ibm-watson-machine-learning-1.0.320:
      Successfully uninstalled ibm-watson-machine-learning-1.0.320


### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [2]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()


Please enter your api key (hit enter): ········


### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

## Define the inference function
In this section we define the inference function. 
Invocation parameters can be modified in the following cell

In [5]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

#### List available models

All avaliable models are presented under `ModelTypes` class.

In [6]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods


print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT']


**If you wish, change model parameters** (don't forget to rerun the cell)

In [7]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 300,
    "min_new_tokens": 50,
    "stop_sequences": ["."],
    "repetition_penalty": 1
}

**If you wish, change the model type for prompting** (don't forget to rerun the cell)

In [8]:
model_id = "meta-llama/llama-2-70b-chat"

## Task 1: General summarization

* Write a prompt to summarize the following customer feedback regarding their loan application experience. If needed, you can use prompt lab for experimentation and bring your final prompt to this notebook. 

* Bonus: The default model is `google/flan-ul2` but is it the best model for this task? Maybe? You can choose another model supported by watsonx.ai. Read the [WML docs](https://ibm.github.io/watson-machine-learning-sdk/index.html) and find the model parameter you need to switch to the model of your choice. Yes, it is an exercise about reading documentaion and python code:) Don't forget that you can adjust other model parameters such as decoding_method, max_new_tokens, min_new_tokens as well!

* Customer feedback:

I hope this message finds you well. I am writing to express my deep disappointment
and frustration regarding my recent loan application experience with your institution.
While I understand that creditworthiness plays a significant role in the approval
process, I am troubled by the lack of communication and transparency throughout the
entire journey.
From the outset, I believed that I had a strong chance of securing the loan I applied for.
However, to my surprise and dismay, my loan application was rejected due to my bad
credit history. What troubles me most is that this crucial information was not
communicated to me by you or any other member of your team. As an applicant, I had
a right to be informed promptly about any setbacks or challenges that could potentially
impact the outcome of my application.
Had I been made aware of the issues with my credit, I could have taken immediate
action to rectify the situation. I would have appreciated an open and honest discussion
about the concerns surrounding my creditworthiness. Instead, I was left in the dark,
unaware that my dreams of obtaining the loan were rapidly fading.
Furthermore, the lack of transparency is not the only aspect of this experience that has
left me disheartened. Throughout the application process, I provided all the required
documentation promptly and diligently. I expected a level of professionalism that
would include timely updates and proactive communication regarding the progress of
my application. Regrettably, I found myself repeatedly following up for information,
only to be met with vague responses and delayed replies.
As a valued customer, I expected better treatment from your institution. The failure to
inform me about the detrimental impact of my bad credit and the overall lack of
communication have not only caused immense frustration but have also wasted
valuable time and effort. I trusted your expertise and guidance throughout this
process, only to be let down by the absence of effective communication channels.
I believe that clear communication is the cornerstone of any successful business
relationship. Had I received timely and accurate updates regarding my loan
application, I would have been able to explore alternative options or address the credit
concerns more proactively. Instead, I was left feeling deceived and abandoned.
I sincerely hope that you will take this feedback seriously and reevaluate your internal
procedures. Improving communication practices and ensuring transparency will not
only enhance the customer experience but also promote trust and loyalty. As a
prospective borrower, I genuinely believe in the value of constructive criticism and
hope that my feedback will contribute to the improvement of your loan application
process.
Thank you for your attention to this matter. I trust that you will take the necessary
steps to address these concerns and prevent such disappointments from happening in
the future.


* sample output:

The customer expresses deep disappointment and frustration with the institution's lack of transparency and communication, particularly in regard to their bad credit history, which ultimately led to the rejection of their loan application, and they hope their feedback will lead to improved procedures and greater trust in the future.


**Replace with your prompt**

In [9]:
prompt_input = """Summarize the customer feedback:

Email:
I hope this message finds you well. I am writing to express my deep disappointment
and frustration regarding my recent loan application experience with your institution.
While I understand that creditworthiness plays a significant role in the approval
process, I am troubled by the lack of communication and transparency throughout the
entire journey.
From the outset, I believed that I had a strong chance of securing the loan I applied for.
However, to my surprise and dismay, my loan application was rejected due to my bad
credit history. What troubles me most is that this crucial information was not
communicated to me by you or any other member of your team. As an applicant, I had
a right to be informed promptly about any setbacks or challenges that could potentially
impact the outcome of my application.
Had I been made aware of the issues with my credit, I could have taken immediate
action to rectify the situation. I would have appreciated an open and honest discussion
about the concerns surrounding my creditworthiness. Instead, I was left in the dark,
unaware that my dreams of obtaining the loan were rapidly fading.
Furthermore, the lack of transparency is not the only aspect of this experience that has
left me disheartened. Throughout the application process, I provided all the required
documentation promptly and diligently. I expected a level of professionalism that
would include timely updates and proactive communication regarding the progress of
my application. Regrettably, I found myself repeatedly following up for information,
only to be met with vague responses and delayed replies.
As a valued customer, I expected better treatment from your institution. The failure to
inform me about the detrimental impact of my bad credit and the overall lack of
communication have not only caused immense frustration but have also wasted
valuable time and effort. I trusted your expertise and guidance throughout this
process, only to be let down by the absence of effective communication channels.
I believe that clear communication is the cornerstone of any successful business
relationship. Had I received timely and accurate updates regarding my loan
application, I would have been able to explore alternative options or address the credit
concerns more proactively. Instead, I was left feeling deceived and abandoned.
I sincerely hope that you will take this feedback seriously and reevaluate your internal
procedures. Improving communication practices and ensuring transparency will not
only enhance the customer experience but also promote trust and loyalty. As a
prospective borrower, I genuinely believe in the value of constructive criticism and
hope that my feedback will contribute to the improvement of your loan application
process.
Thank you for your attention to this matter. I trust that you will take the necessary
steps to address these concerns and prevent such disappointments from happening in
the future.

Summary: """

In [10]:
# Run the test
prompt = Prompt(access_token, project_id)
prompt.generate(prompt_input, model_id, parameters)

'\nThe customer is unhappy with their recent loan application experience with the institution. They claim that there was a lack of communication and transparency throughout the process, particularly regarding their bad credit history, which ultimately led to their loan application being rejected.'

## Task 2: Targeted summarization

* Write a prompt that summarize both positive and negative aspects in the following customer feedback regarding their loan application experience.

* Bonus: List the summary in bullet points.

* Customer feedback you will work with:

I started my loan process toward securing a VA loan. I was waiting for a a month and a
couple weeks, then I was told that the VA needed to acquire my retirement points to
verify my veteran status. If I knew this is what my loan was on hold for, I could have
contacted the VA office right away and got this cleared up. For whatever reason, it took
the underwriting department a long time to verify my employment status, even after I
uploaded a couple years of my W2 forms from both of my jobs, and they had my Social
Security number to further verify my employment status. My loan completion date was
extended, because I wasn't made aware that they were waiting for my VA status to be
approved. The push back for my mortgage is common for mortgage companies, but this
caused my interest rate to go up. Then, the securing of a closing lawyer being made
aware to me and the lawyer needing three days to get their end prepared for me to go
to their office to sign the paperwork wasn't made aware to me. My loan missed the
second closing date. For whatever reason, the locked in interest rate jumped up 5/8
points. After making the banker I was working with aware of this, he didn't understand
why the locked in interest rate jumped up either. He was nice enough to work on it and
was able to get the interest rate down in 1/4 of a point, so my mortgage has an interest
rate that is 3/8 of a point higher than my locked in interest rate in the beginning of this
process. Although my interest rate is higher than the locked in interest point, at the
end, the mortgage is successfully finished.

* Sample output:

```
* Positive aspects:
	+ The banker was nice and helpful in resolving the issue with the interest rate.
	+ The loan was successfully finished despite the challenges.
* Negative aspects:
	+ The loan process took longer than expected due to delays in verifying the customer's veteran status and employment status.
	+ The customer was not kept informed about the status of their loan and was not aware of the need for a closing lawyer.
	+ The interest rate increased by 5/8 of a point, resulting in a higher interest rate than the customer had locked in.
	+ The customer had to work with the banker to get the interest rate reduced by 1/4 of a point.

In [11]:
prompt_input = """Summarize the following customer feedback and focus on both positive and negative aspects in bullet points. 

Email:
I started my loan process toward securing a VA loan. I was waiting for a a month and a
couple weeks, then I was told that the VA needed to acquire my retirement points to
verify my veteran status. If I knew this is what my loan was on hold for, I could have
contacted the VA office right away and got this cleared up. For whatever reason, it took
the underwriting department a long time to verify my employment status, even after I
uploaded a couple years of my W2 forms from both of my jobs, and they had my Social
Security number to further verify my employment status. My loan completion date was
extended, because I wasn't made aware that they were waiting for my VA status to be
approved. The push back for my mortgage is common for mortgage companies, but this
caused my interest rate to go up. Then, the securing of a closing lawyer being made
aware to me and the lawyer needing three days to get their end prepared for me to go
to their office to sign the paperwork wasn't made aware to me. My loan missed the
second closing date. For whatever reason, the locked in interest rate jumped up 5/8
points. After making the banker I was working with aware of this, he didn't understand
why the locked in interest rate jumped up either. He was nice enough to work on it and
was able to get the interest rate down in 1/4 of a point, so my mortgage has an interest
rate that is 3/8 of a point higher than my locked in interest rate in the beginning of this
process. Although my interest rate is higher than the locked in interest point, at the
end, the mortgage is successfully finished.

Summary: """

In [12]:
# Run the test
prompt = Prompt(access_token, project_id)
prompt.generate(prompt_input, model_id, parameters)

'\n\n* Positive aspects:\n\t+ The customer was able to secure a VA loan.\n\t+ The banker was nice and helpful in resolving the issue with the interest rate.\n\t+ The mortgage was successfully finished.'

## Task 3: Targeted summarization focusing interest rate

* You will be working with the same customer feedback from Task 2. But this time you will write a prompt that generates a summary focusing on **interest rate**.


* Customer feedback you will work with:

I started my loan process toward securing a VA loan. I was waiting for a a month and a
couple weeks, then I was told that the VA needed to acquire my retirement points to
verify my veteran status. If I knew this is what my loan was on hold for, I could have
contacted the VA office right away and got this cleared up. For whatever reason, it took
the underwriting department a long time to verify my employment status, even after I
uploaded a couple years of my W2 forms from both of my jobs, and they had my Social
Security number to further verify my employment status. My loan completion date was
extended, because I wasn't made aware that they were waiting for my VA status to be
approved. The push back for my mortgage is common for mortgage companies, but this
caused my interest rate to go up. Then, the securing of a closing lawyer being made
aware to me and the lawyer needing three days to get their end prepared for me to go
to their office to sign the paperwork wasn't made aware to me. My loan missed the
second closing date. For whatever reason, the locked in interest rate jumped up 5/8
points. After making the banker I was working with aware of this, he didn't understand
why the locked in interest rate jumped up either. He was nice enough to work on it and
was able to get the interest rate down in 1/4 of a point, so my mortgage has an interest
rate that is 3/8 of a point higher than my locked in interest rate in the beginning of this
process. Although my interest rate is higher than the locked in interest point, at the
end, the mortgage is successfully finished.

* Sample output (Note: your output may vary)

```
The customer experienced multiple delays in the loan process, which ultimately caused the locked-in interest rate to increase by 5/8 points. However, after raising the issue, the banker managed to reduce it by 1/4 of a point, resulting in a final interest rate that was 3/8 of a point higher than the initial agreement, but the mortgage was successfully completed.
```


In [13]:
prompt_input = """Summarize what happened to the interest rate from the customer feedback.

Customer feedback:

I started my loan process toward securing a VA loan. I was waiting for a a month and a
couple weeks, then I was told that the VA needed to acquire my retirement points to
verify my veteran status. If I knew this is what my loan was on hold for, I could have
contacted the VA office right away and got this cleared up. For whatever reason, it took
the underwriting department a long time to verify my employment status, even after I
uploaded a couple years of my W2 forms from both of my jobs, and they had my Social
Security number to further verify my employment status. My loan completion date was
extended, because I wasn't made aware that they were waiting for my VA status to be
approved. The push back for my mortgage is common for mortgage companies, but this
caused my interest rate to go up. Then, the securing of a closing lawyer being made
aware to me and the lawyer needing three days to get their end prepared for me to go
to their office to sign the paperwork wasn't made aware to me. My loan missed the
second closing date. For whatever reason, the locked in interest rate jumped up 5/8
points. After making the banker I was working with aware of this, he didn't understand
why the locked in interest rate jumped up either. He was nice enough to work on it and
was able to get the interest rate down in 1/4 of a point, so my mortgage has an interest
rate that is 3/8 of a point higher than my locked in interest rate in the beginning of this
process. Although my interest rate is higher than the locked in interest point, at the
end, the mortgage is successfully finished.

Summary:"""

In [14]:
# Run the test
prompt = Prompt(access_token, project_id)
prompt.generate(prompt_input, model_id, parameters)

"\n\nThe customer's interest rate increased by 5/8 points due to delays in the loan process, specifically in the underwriting department, and the customer was not made aware of the delay in securing a closing lawyer. The banker was able to lower the interest rate by 1/4 of a point, resulting in a final interest rate that is 3/8 of a point higher than the original locked-in rate."

## Task 4: Sentiment Analysis

* Use an IBM model and write a prompt to return the sentiment (positive or negative) of the following reviews:

**Customer feedback 1:**

I started my loan process toward securing a VA loan. I was waiting for a a month and a
couple weeks, then I was told that the VA needed to acquire my retirement points to
verify my veteran status. If I knew this is what my loan was on hold for, I could have
contacted the VA office right away and got this cleared up. For whatever reason, it took
the underwriting department a long time to verify my employment status, even after I
uploaded a couple years of my W2 forms from both of my jobs, and they had my Social
Security number to further verify my employment status. My loan completion date was
extended, because I wasn't made aware that they were waiting for my VA status to be
approved. The push back for my mortgage is common for mortgage companies, but this
caused my interest rate to go up. Then, the securing of a closing lawyer being made
aware to me and the lawyer needing three days to get their end prepared for me to go
to their office to sign the paperwork wasn't made aware to me. My loan missed the
second closing date. For whatever reason, the locked in interest rate jumped up 5/8
points. After making the banker I was working with aware of this, he didn't understand
why the locked in interest rate jumped up either. He was nice enough to work on it and
was able to get the interest rate down in 1/4 of a point, so my mortgage has an interest
rate that is 3/8 of a point higher than my locked in interest rate in the beginning of this
process. Although my interest rate is higher than the locked in interest point, at the
end, the mortgage is successfully finished.

**Customer Feedback 2:**

I hope this message finds you well. I am writing to express my deep disappointment
and frustration regarding my recent loan application experience with your institution.
While I understand that creditworthiness plays a significant role in the approval
process, I am troubled by the lack of communication and transparency throughout the
entire journey.
From the outset, I believed that I had a strong chance of securing the loan I applied for.
However, to my surprise and dismay, my loan application was rejected due to my bad
credit history. What troubles me most is that this crucial information was not
communicated to me by you or any other member of your team. As an applicant, I had
a right to be informed promptly about any setbacks or challenges that could potentially
impact the outcome of my application.
Had I been made aware of the issues with my credit, I could have taken immediate
action to rectify the situation. I would have appreciated an open and honest discussion
about the concerns surrounding my creditworthiness. Instead, I was left in the dark,
unaware that my dreams of obtaining the loan were rapidly fading.
Furthermore, the lack of transparency is not the only aspect of this experience that has
left me disheartened. Throughout the application process, I provided all the required
documentation promptly and diligently. I expected a level of professionalism that
would include timely updates and proactive communication regarding the progress of
my application. Regrettably, I found myself repeatedly following up for information,
only to be met with vague responses and delayed replies.
As a valued customer, I expected better treatment from your institution. The failure to
inform me about the detrimental impact of my bad credit and the overall lack of
communication have not only caused immense frustration but have also wasted
valuable time and effort. I trusted your expertise and guidance throughout this
process, only to be let down by the absence of effective communication channels.
I believe that clear communication is the cornerstone of any successful business
relationship. Had I received timely and accurate updates regarding my loan
application, I would have been able to explore alternative options or address the credit
concerns more proactively. Instead, I was left feeling deceived and abandoned.
I sincerely hope that you will take this feedback seriously and reevaluate your internal
procedures. Improving communication practices and ensuring transparency will not
only enhance the customer experience but also promote trust and loyalty. As a
prospective borrower, I genuinely believe in the value of constructive criticism and
hope that my feedback will contribute to the improvement of your loan application
process.

Thank you for your attention to this matter. I trust that you will take the necessary
steps to address these concerns and prevent such disappointments from happening in
the future.

**Customer feedback 3:**

I recently went through the home loan process, and I must say it was a smooth and
efficient experience. From the initial application to the final closing, the loan officer
guided me every step of the way, ensuring all my questions were answered. The
communication was excellent, with timely updates on the progress of my loan. The
team was highly professional and dedicated to getting me the best possible terms.
They worked diligently to secure a competitive interest rate, and the loan was
completed within the expected timeframe. I am extremely satisfied with the service
provided and would highly recommend their home loan services to anyone in need of
financing.

* Expected output:
```
Negative
negative
positive
```



In [20]:
reviews = f"""
    
Review 1: I started my loan process toward securing a VA loan. I was waiting for a a month and a
couple weeks, then I was told that the VA needed to acquire my retirement points to
verify my veteran status. If I knew this is what my loan was on hold for, I could have
contacted the VA office right away and got this cleared up. For whatever reason, it took
the underwriting department a long time to verify my employment status, even after I
uploaded a couple years of my W2 forms from both of my jobs, and they had my Social
Security number to further verify my employment status. My loan completion date was
extended, because I wasn't made aware that they were waiting for my VA status to be
approved. The push back for my mortgage is common for mortgage companies, but this
caused my interest rate to go up. Then, the securing of a closing lawyer being made
aware to me and the lawyer needing three days to get their end prepared for me to go
to their office to sign the paperwork wasn't made aware to me. My loan missed the
second closing date. For whatever reason, the locked in interest rate jumped up 5/8
points. After making the banker I was working with aware of this, he didn't understand
why the locked in interest rate jumped up either. He was nice enough to work on it and
was able to get the interest rate down in 1/4 of a point, so my mortgage has an interest
rate that is 3/8 of a point higher than my locked in interest rate in the beginning of this
process. Although my interest rate is higher than the locked in interest point, at the
end, the mortgage is successfully finished.

Review 2: I hope this message finds you well. I am writing to express my deep disappointment
and frustration regarding my recent loan application experience with your institution.
While I understand that creditworthiness plays a significant role in the approval
process, I am troubled by the lack of communication and transparency throughout the
entire journey.
From the outset, I believed that I had a strong chance of securing the loan I applied for.
However, to my surprise and dismay, my loan application was rejected due to my bad
credit history. What troubles me most is that this crucial information was not
communicated to me by you or any other member of your team. As an applicant, I had
a right to be informed promptly about any setbacks or challenges that could potentially
impact the outcome of my application.
Had I been made aware of the issues with my credit, I could have taken immediate
action to rectify the situation. I would have appreciated an open and honest discussion
about the concerns surrounding my creditworthiness. Instead, I was left in the dark,
unaware that my dreams of obtaining the loan were rapidly fading.
Furthermore, the lack of transparency is not the only aspect of this experience that has
left me disheartened. Throughout the application process, I provided all the required
documentation promptly and diligently. I expected a level of professionalism that
would include timely updates and proactive communication regarding the progress of
my application. Regrettably, I found myself repeatedly following up for information,
only to be met with vague responses and delayed replies.
As a valued customer, I expected better treatment from your institution. The failure to
inform me about the detrimental impact of my bad credit and the overall lack of
communication have not only caused immense frustration but have also wasted
valuable time and effort. I trusted your expertise and guidance throughout this
process, only to be let down by the absence of effective communication channels.
I believe that clear communication is the cornerstone of any successful business
relationship. Had I received timely and accurate updates regarding my loan
application, I would have been able to explore alternative options or address the credit
concerns more proactively. Instead, I was left feeling deceived and abandoned.
I sincerely hope that you will take this feedback seriously and reevaluate your internal
procedures. Improving communication practices and ensuring transparency will not
only enhance the customer experience but also promote trust and loyalty. As a
prospective borrower, I genuinely believe in the value of constructive criticism and
hope that my feedback will contribute to the improvement of your loan application
process. Thank you for your attention to this matter. I trust that you will take the necessary
steps to address these concerns and prevent such disappointments from happening in
the future.

Review 3: I recently went through the home loan process, and I must say it was a smooth and
efficient experience. From the initial application to the final closing, the loan officer
guided me every step of the way, ensuring all my questions were answered. The
communication was excellent, with timely updates on the progress of my loan. The
team was highly professional and dedicated to getting me the best possible terms.
They worked diligently to secure a competitive interest rate, and the loan was
completed within the expected timeframe. I am extremely satisfied with the service
provided and would highly recommend their home loan services to anyone in need of
financing."""


prompt_input = f"""Classify sentiment for Review 1, Review 2, and Review 3 to positive or negative. Provide output in this format: Review 1: sentiment, Review 2: sentiment.

                Reviews:
                {reviews}
    
                Sentiment:
                """


In [21]:
# Run the test
prompt = Prompt(access_token, project_id)
prompt.generate(prompt_input, model_id, parameters)

'\nReview 1: Negative\nReview 2: Negative\nReview 3: Positive\n\nThe sentiment of Review 1 is negative because the reviewer expresses frustration with the loan process, specifically with the lack of communication and transparency from the lender.'

## Task 5: Named entity Extraction

* Please use an IBM model for this task and write a prompt to extract details about the insured vehicle from the auto insturance claim report below. The expected output should in JSON format that looks like this:
```
{
    "Car Make and Model": "Tesla Model S",
    "Location": "Astor Place, New York",
    "Date": "April 15th, 2023",
    "Time": "Not Found"
}

```

* Customer feedback you will work with:

The insured vehicle, a Tesla Model S, was parked outside on Astor Place, New York on April 15th, 2023 when an unexpected and violent hailstorm struck the area.\n Hailstones of varying sizes, ranging from small pellets to large golf balls, rained down on the vehicle with relentless force. The impact of the hailstones caused extensive damage to the exterior of the vehicle, leaving behind a multitude of dents and dings on the roof, hood, trunk, and all sides.\n The hailstones were so powerful that they shattered the windshield and side windows, resulting in shattered glass fragments scattered throughout the interior of the vehicle. The insured immediately contacted their insurance company to report the incident and initiated the claims process.\n Recognizing the severity of the damage, the insurance company promptly dispatched an experienced auto appraiser to assess the extent of the hail damage.\nThe appraiser meticulously inspected the vehicle, documenting each dent, ding, and broken window. Based on their assessment, the appraiser estimated the cost of repairs, taking into account the need for paintless dent removal, panel replacements, windshield replacement, and window repairs. With the appraisal report in hand, the insured is now filing a comprehensive claim for the necessary repairs.\n Along with the report, the insured is submitting detailed photographs of the damaged vehicle from multiple angles, capturing the extent of the hail damage.\nThe insured understands the importance of providing compelling evidence to support their claim and has meticulously documented the damage. They are also including any additional information or documentation requested by the insurance company. The insured is eagerly awaiting the outcome of their claim, hopeful that the insurance company will process it promptly.\nThey recognize that the repairs required to restore their beloved Tesla Model S to its pre-hailstorm condition will be extensive and costly. However, they have confidence in the insurance company's commitment to providing the necessary coverage and support. Their ultimate goal is to see their vehicle restored to its original beauty and performance, ready to hit the road once again, unaffected by the devastating hailstorm.

* <span style="color:red">Hint: You can try few-shot prompting using sample customer feedback in the sample_insurance_claims.txt file. If you did not download the whole repo for the lab, you can find it [here](https://ibm.box.com/s/sj4cxdu05bv9onyxzl43ye0w8vd4xhmk)</span>.


In [25]:
prompt_input = """Extract car make and model, location, date and time from the insurance claim description below.  If you don't find these details in the description, please fill it as Not Found. Format the output in JSON format.

Insurance claim description: 
A car accident occurred on Jan 1st, 2023 at 5pm at the intersection of woodbridge. The insured vehicle, a Honda Civic, was hit by another vehicle that ran a red light. The insured driver, John, was driving within the speed limit and following all traffic rules. The accident resulted in significant damage to the insured vehicle, including a broken bumper and damaged front fender. There were no injuries reported. The insured is filing a claim for the repairs and any necessary medical expenses. 

Output:
{Car make and model: Honda Civic; Location: Woodbridge; Date: Jan 1st, 2023; Time: 5pm}

Insurance claim description: 
The insured vehicle, a Ford RAM, was stolen from Boston on Dec 2nd 2022. The vehicle was parked in a secure parking lot, and all necessary precautions were taken, such as locking the doors and activating the alarm system. The insured immediately reported the theft to the police and obtained a police report. The vehicle had comprehensive insurance coverage, and the insured is filing a claim for the stolen vehicle, including its estimated value, accessories, and personal belongings that were inside the vehicle at the time of theft. 

Output:
{Car make and model: Ford RAM; Location: Boston; Date: Dec 2nd 2022; Time: Not Found}

Insurance claim description: 
The insured vehicle, a Tesla model X, was vandalized on march 23rd while parked in front of the insured's residence on Magador Street. The vandalism included scratched paint, broken windows, and damage to the side mirrors. The insured promptly reported the incident to the police and obtained a police report. The insured is filing a claim for the repairs and any necessary replacement parts. The estimated cost of repairs has been assessed by a reputable auto repair shop. 

Output:
{Car make and model: Tesla Model X; Location: Magador Street; Date: march 23rd; Time: Not Found}

Insurance claim description: 
The insured vehicle, a Tesla Model S, was parked outside on Astor Place, New York on April 15th, 2023 when an unexpected and violent hailstorm struck the area.\n Hailstones of varying sizes, ranging from small pellets to large golf balls, rained down on the vehicle with relentless force. The impact of the hailstones caused extensive damage to the exterior of the vehicle, leaving behind a multitude of dents and dings on the roof, hood, trunk, and all sides.\n The hailstones were so powerful that they shattered the windshield and side windows, resulting in shattered glass fragments scattered throughout the interior of the vehicle. The insured immediately contacted their insurance company to report the incident and initiated the claims process.\n Recognizing the severity of the damage, the insurance company promptly dispatched an experienced auto appraiser to assess the extent of the hail damage.\nThe appraiser meticulously inspected the vehicle, documenting each dent, ding, and broken window. Based on their assessment, the appraiser estimated the cost of repairs, taking into account the need for paintless dent removal, panel replacements, windshield replacement, and window repairs. With the appraisal report in hand, the insured is now filing a comprehensive claim for the necessary repairs.\n Along with the report, the insured is submitting detailed photographs of the damaged vehicle from multiple angles, capturing the extent of the hail damage.\nThe insured understands the importance of providing compelling evidence to support their claim and has meticulously documented the damage. They are also including any additional information or documentation requested by the insurance company. The insured is eagerly awaiting the outcome of their claim, hopeful that the insurance company will process it promptly.\nThey recognize that the repairs required to restore their beloved Tesla Model S to its pre-hailstorm condition will be extensive and costly. However, they have confidence in the insurance company's commitment to providing the necessary coverage and support. Their ultimate goal is to see their vehicle restored to its original beauty and performance, ready to hit the road once again, unaffected by the devastating hailstorm.

Output:
"""

In [26]:
# Run the test
prompt = Prompt(access_token, project_id)
prompt.generate(prompt_input, model_id, parameters)

'{Car make and model: Tesla Model S; Location: Astor Place, New York; Date: April 15th, 2023; Time: Not Found}\n\nInsurance claim description: \nThe insured vehicle, a Toyota Camry, was involved in a hit-and-run accident on June 12th, 2023, at 10:30 am.'

## Task 6: Question and Answering

* Write a prompt to get the model to answer a question based on the following Exclusions section from an auto insurance policy. 

```
The Insurance Company will not pay for any damages an insured
person is legally obligated to pay because of:

1. bodily injury or property damage arising out of
the use of your insured auto while used to carry
persons or property for a charge, or any auto
you are driving while available for hire by the
public to the extent that the limits of liability for
this coverage exceed the limits of liability
required by the Nevada financial responsibility
law. This exclusion does not apply to shared
expense car pools.

2. bodily injury or property damage arising out of
the use of a non-owned auto in any business or
occupation of an insured person to the extent
that the limits of liability for this coverage exceed
the limits of liability required by the Nevada
financial responsibility law. However, this
exclusion does not apply while you, your
chauffeur, or domestic servant are using a
private passenger auto or trailer.

3. bodily injury to an employee of any insured
person arising in the course of employment.
This exclusion does not apply to your domestic employee 
who is not required to be covered by
a workers compensation law or similar law.

4. bodily injury to a co-worker injured in the
course of employment. This exclusion does not
apply to you.

5. bodily injury to you or any resident of your
household related to you by blood, marriage, or
adoption to the extent that the limits of liability
for this coverage exceed the limits of liability
required by the Nevada financial responsibility
law.

6. damage to or destruction of property an insured
person owns, transports, is in charge of, or
rents. However, a private residence or a garage
rented by that person is covered.

7. bodily injury or property damage which may
reasonably be expected to result from the
intentional acts of an insured person or which
are in fact intended by an insured person to the
extent that the limits of liability for this coverage
exceed the limits of liability required by the
Nevada financial responsibility law.

8. bodily injury or property damage which would
also be covered under nuclear energy liability
insurance to the extent that the limits of liability
for this coverage exceed the limits of liability
required by the Nevada financial responsibility
law. This applies even if the limits of that
insurance are exhausted.
```
* You can come up with your own questions, here are some examples:
```
- Will this policy cover me if I use my personal vehicle for business purposes?
- Does this policy cover me if I get into an accident while driving for a ride-sharing company?
- Does this policy cover injuries to my family members?
```

In [27]:
context = """
Insurance Company will not pay for any damages an insured
person is legally obligated to pay because of:

1. bodily injury or property damage arising out of
the use of your insured auto while used to carry
persons or property for a charge, or any auto
you are driving while available for hire by the
public to the extent that the limits of liability for
this coverage exceed the limits of liability
required by the Nevada financial responsibility
law. This exclusion does not apply to shared
expense car pools.

2. bodily injury or property damage arising out of
the use of a non-owned auto in any business or
occupation of an insured person to the extent
that the limits of liability for this coverage exceed
the limits of liability required by the Nevada
financial responsibility law. However, this
exclusion does not apply while you, your
chauffeur, or domestic servant are using a
private passenger auto or trailer.

3. bodily injury to an employee of any insured
person arising in the course of employment.
This exclusion does not apply to your domestic employee who is not required to be covered by
a workers compensation law or similar law.

4. bodily injury to a co-worker injured in the
course of employment. This exclusion does not
apply to you.

5. bodily injury to you or any resident of your
household related to you by blood, marriage, or
adoption to the extent that the limits of liability
for this coverage exceed the limits of liability
required by the Nevada financial responsibility
law.

6. damage to or destruction of property an insured
person owns, transports, is in charge of, or
rents. However, a private residence or a garage
rented by that person is covered.

7. bodily injury or property damage which may
reasonably be expected to result from the
intentional acts of an insured person or which
are in fact intended by an insured person to the
extent that the limits of liability for this coverage
exceed the limits of liability required by the
Nevada financial responsibility law.

8. bodily injury or property damage which would
also be covered under nuclear energy liability
insurance to the extent that the limits of liability
for this coverage exceed the limits of liability
required by the Nevada financial responsibility
law. This applies even if the limits of that
insurance are exhausted."""

quesiton = "Does this policy cover injuries to my family members?"

# Note: the <<SYS>> format is specific to llama model

prompt_input = f"""<<SYS>>
Context information is below.
------------------------------
{context}
------------------------------

Given the context information answer the following question (if you don't know the answer, use the best of your knowledge) 

<<SYS>>

[INST]{quesiton}[/INST]
"""

In [28]:
# Run the test
prompt = Prompt(access_token, project_id)
prompt.generate(prompt_input, model_id, parameters)

'\nBased on the context information provided, the policy excludes coverage for bodily injury to you or any resident of your household related to you by blood, marriage, or adoption to the extent that the limits of liability for this coverage exceed the limits of liability required by the Nevada financial responsibility law.'

### Authors: 
 **Catherine Cao and Elena Lowery